<a href="https://colab.research.google.com/github/annie-lyu/word-embedding/blob/master/ldaseq_csStack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from gensim import corpora
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
import numpy
from gensim.matutils import hellinger
import nltk


In [3]:
import glob
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
## preparation for text preprocessing
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
stop_words = list(set(stopwords.words('english') + list(string.ascii_lowercase)))
stop_words = stop_words + ["would", "could", "use", "also", "might", "one", "whether", "like", "first", "code", "given"]
stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['k',
 'no',
 'yourself',
 'by',
 'at',
 'too',
 "that'll",
 "wasn't",
 "haven't",
 "won't",
 'on',
 'very',
 'before',
 'how',
 'hasn',
 'both',
 'because',
 'about',
 'during',
 'so',
 'won',
 "shouldn't",
 'here',
 'to',
 'of',
 're',
 'o',
 'for',
 'can',
 'doesn',
 'being',
 'their',
 'which',
 'but',
 "shan't",
 'when',
 'r',
 'i',
 'if',
 'few',
 'g',
 'h',
 'hers',
 'while',
 "needn't",
 'down',
 'theirs',
 'we',
 'has',
 "hasn't",
 'shan',
 'below',
 'll',
 'yours',
 'who',
 'with',
 've',
 'up',
 "hadn't",
 'j',
 'herself',
 'v',
 'your',
 'over',
 'hadn',
 'n',
 'x',
 "she's",
 'were',
 'have',
 'all',
 'ain',
 'an',
 "wouldn't",
 's',
 'as',
 'q',
 'against',
 "didn't",
 'z',
 'them',
 'himself',
 "mightn't",
 'did',
 'weren',
 'f',
 'u',
 'now',
 "should've",
 'couldn',
 'wasn',
 "you'd",
 'once',
 'his',
 'was',
 "isn't",
 'just',
 'should',
 'him',
 'until',
 "it's",
 'you',
 'why',
 'itself',
 'they',
 'having',
 'l',
 'through',
 'w',
 'it',
 'y',
 'don',
 'my',
 'is',

In [5]:
import re
def removePunctuation(text):
    punctuation = '\\\\!,;:?"\'.-=^/|+-'
    text = re.sub(r'[{}]+'.format(punctuation),'',text)
    text = text.replace("\n", "")
    return text

In [6]:
lines =  [line.lower() for line in open("/content/drive/My Drive/words/cs-stack.txt",'r', errors='ignore')]
posts = [removePunctuation(line) for line in lines]
posts[1]


'consider the following scenario as shown in imagei have summarized the above memory layout in terms of miss rates and hit rates                                                                               lmr         gmr         lhr         ghr       l cache                              l cache                                    main memory                                   sum of columns  not useful  not useful  not useful        here lmr  local miss rate gmr  global miss rate lhr  local hit rate ghr  global hit ratefrom the above table i deduced the following insightssum of the global hit rates (ghr) of all caches and main memory is the formula for amat(average memory access time) can be interpreted as follows  amat  t  mt   mmt(or)   amat  t  mt   mtwhere mm are local miss rates of level  and level  caches respectively mm are global miss rates of level  and level  caches respectively t t t are access times for level  cache level  cache and main memory for the given example   amat 

In [7]:
## establish dictionary and remove stop words
dictionary_cs= corpora.Dictionary(post.split() for post in posts)
stop_ids = [dictionary_cs.token2id[stopword] for stopword in stop_words if stopword in dictionary_cs.token2id]
dictionary_cs.filter_tokens(stop_ids)
dictionary_cs.compactify()
dictionary_cs.save("/content/drive/My Drive/words/stack_dictionary")


## create vocabulary file
class MyCorpus(object):
    def __iter__(self):
        for line in open("/content/drive/My Drive/words/cs-stack.txt", errors= "ignore"):
            yield dictionary_cs.doc2bow(line.lower().split())

corpus_memory_friendly_cs = MyCorpus()
corpus_cs = [vector for vector in corpus_memory_friendly_cs]
corpora.BleiCorpus.serialize('/content/drive/My Drive/words/corpus_cs', corpus_cs)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [8]:
## loading and setting
try:
  dictionary_cs = Dictionary.load("/content/drive/My Drive/words/stack_dictionary")
except FileNotFoundError as e:
  raise ValueError("SKIP: Please download the Corpus/news_dictionary dataset.")
corpus_cs = bleicorpus.BleiCorpus('/content/drive/My Drive/words/corpus_cs')

time_slice = [6000]



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
ldaseq_cs_5 = ldaseqmodel.LdaSeqModel(corpus=corpus_cs, id2word=dictionary_cs, time_slice = time_slice, num_topics=5)
ldaseq_cs_10 = ldaseqmodel.LdaSeqModel(corpus=corpus_cs, id2word=dictionary_cs, time_slice = time_slice, num_topics=10)



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/models/ldaseqmodel.py:1472: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.fabs((lhood_old - lhood) / (lhood_old * total))
/usr/local/lib/python3.6/dist-packages/gensim/models/ldaseqmodel.py:290: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migr

In [10]:
#Printing Topics
ldaseq_cs_5.print_topics(time=0)

[[('type', 0.008551586936877398),
  ('programming', 0.005380449392343622),
  ('function', 0.0049630332983396416),
  ('data', 0.004746041179704067),
  ('language', 0.0046073868557377036),
  ('computer', 0.004383162707042253),
  ('lambda', 0.003886563554278372),
  ('using', 0.0037644577591066826),
  ('system', 0.0037472955226271333),
  ('different', 0.0037282073776191164),
  ('used', 0.003692730870361659),
  ('languages', 0.0036101600448354833),
  ('may', 0.0035412099050915716),
  ('program', 0.0035193402177931656),
  ('want', 0.0035062703399423903),
  ('way', 0.003486300962110543),
  ('many', 0.003467445077716834),
  ('know', 0.00316228857935635),
  ('types', 0.003066657974944906),
  ('algorithms', 0.0029385214297008044)],
 [('time', 0.012635898135965043),
  ('number', 0.009248691848019594),
  ('data', 0.008613016468107788),
  ('memory', 0.0067701619300003886),
  ('function', 0.005099291114154077),
  ('return', 0.004993897746862989),
  ('array', 0.004926777618286154),
  ('two', 0.004665

In [11]:
ldaseq_cs_10.print_topics(time=0)

[[('time', 0.022001823998658798),
  ('algorithm', 0.013952431368047212),
  ('array', 0.009618564027597552),
  ('number', 0.009134784802046249),
  ('elements', 0.008441963205036802),
  ('element', 0.008073024636474819),
  ('function', 0.0075487074788944935),
  ('n)', 0.006838585310058362),
  ('case', 0.006633318159805173),
  ('value', 0.006491739832630546),
  ('complexity', 0.006414951613121042),
  ('log', 0.006263309890699951),
  ('find', 0.006155956844890464),
  ('list', 0.0058576525871056545),
  ('size', 0.005562005402087663),
  ('hash', 0.005545321194150981),
  ('running', 0.005464763004834058),
  ('need', 0.005320318174028618),
  ('data', 0.005292363088544393),
  ('two', 0.005021492279519186)],
 [('number', 0.023173704421806402),
  ('numbers', 0.015796017492798086),
  ('two', 0.008392541572922825),
  ('bits', 0.0082848589546738),
  ('sum', 0.008040798605444193),
  ('random', 0.007153386990249172),
  ('bit', 0.006786467575589071),
  ('algorithm', 0.006659994224624456),
  ('probabili